<a href="https://colab.research.google.com/github/MicroprocessorX069/Collaborative-Filtering-for-medical-history/blob/master/Skip_n_gram_CF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

TO DO


1.   Plot Model alphabet , specified no.
2.   Optimize model
3.   Create a pipeline for prediction and implement it



##Reading the history of patients


##Reading the histo

In [0]:
import pandas as pd
df=pd.read_csv("fcodesbucket.csv",header=None)
df=df.iloc[1:,1:]
import math
#adjacent elements of original dataset added here in df_adj
df_adj=pd.DataFrame()
no_past_symptoms=2
for x in range(len(df)):
  y=0
  while y+no_past_symptoms<len(df.iloc[x]) and not(pd.isna(df.iloc[x,y+no_past_symptoms])):
    temp=list(df.iloc[x,y:y+no_past_symptoms])
    df2 = pd.DataFrame([temp])
    df_adj=df_adj.append(df2)
    y+=1
#df_adj=df_adj.reset_index() # to make the indices of rows as 0,1,2,3...
df_adj=df_adj.dropna()
df_adj

36765 rows, 2 columns  (26927 unique combinations; 73.24 %)

Each column has 8.87%, 8.73% of unique codes, respectively.


---



48.761 % times the letter associated with adjacent symptoms are adjcant alphabetically

36.36 % times the letters are same

In [0]:
#separates letters and numbers for each fcode.

df_adj=df_adj.dropna()
df_expanded=pd.DataFrame()
df_k=pd.DataFrame()
for el1,el2 in zip(df_adj[0],df_adj[1]):  
  if (el1[1:].isdigit() and el2[1:].isdigit()):
    df2 = pd.DataFrame([[   el1[0],str(int(el1[1:])),el2[0],str(int(el2[1:]))  ]])
    df_expanded=pd.concat([df_expanded,df2])
df_expanded=df_expanded.rename(columns={0:"l_pCode",1:"l_cCode",2:"r_pCode",3:"r_cCode"})
#print(count*100/len(df_adj))

Symptoms with 'K' have the maximum occurences

8461 Occurences in 37k

Normalizing the dataset


3110 unique codes(x) in 32016 pairs

Nomenclature for the dataset

All variable named with underscore
All functions named with Camel Casing
1.   Code on left: x_Code; right: y_Code, unique_xCode, n_unique_xCodes
2.   parent_code: x_pCode; y_pCode, unique_x_pCode, n_unique_x_pCode
3.   Child: x_cCode; y_cCode
4. 




##Set creation

In [0]:
unique_l_pCode=list(df_expanded.iloc[:,0].drop_duplicates())
n_unique_l_pCode=len(unique_l_pCode)
unique_l_cCode=list(df_expanded.iloc[:,1].drop_duplicates())
n_unique_l_cCode=len(unique_l_cCode)
unique_r_pCode=list(df_expanded.iloc[:,2].drop_duplicates())
n_unique_r_pCode=len(unique_r_pCode)

print(n_unique_l_cCode)
df_llr=df_expanded.iloc[:,0:3]
df_llr=df_llr.rename(columns={0:"l_pCode",1:"l_cCode",2:"r_pCode"})

##Splitting Data respect to each attribute

In [0]:
h=df_adj.shape[1]
v=df_adj.shape[0]
n_unique=len(df_adj.iloc[:,0].drop_duplicates())
print(n_unique)
df_sorted=[]
for i in range(n_unique):
  searchCode=df_adj.iloc[i,0]
  processedData=[]
  processedLabel=[]
  temp=[]
  for j in range(v):
    if df_adj.iloc[j,0]==searchCode:
      processedData.append(searchCode)
      processedLabel.append(df_adj.iloc[j,1])
      temp.append([searchCode,df_adj.iloc[j,1]])
      
  df_sorted.append(temp)
df_sorted    

In [0]:
## Saving as csv (Optional)
import pandas as pd 
pd.DataFrame(np.array(df_sorted)).to_csv("df_sorted.csv")

In [0]:
import pandas as pd
df_sorted=pd.read_csv("df_sorted.csv",header=None)
df_sorted
#df_so=df.iloc[1:,1:]

32016 rows,

3110 unique symptoms in x

3069 unique symptoms in y

In [0]:
df_norm = (df_expanded - df_expanded.mean()) / (df_expanded.max() - df_expanded.min())

##Model 1 : Embeddings for parent codes

Setting up the data

X has a parent code and child code
Y has a parent code

All values are categorical hence converted to one hot encodings

In [0]:

processedData=pd.get_dummies(df_llr.iloc[:,0:2])
processedLabel=pd.get_dummies(df_llr.iloc[:,2])
processedData.shape
processedLabel.shape

(32012, 20)

In [0]:
processedData.iloc[0]

In [0]:
import tensorflow as tf
import numpy as np
pAttributes=[]
for unique_pCode in unique_l_pCode:
  print("For unique code: "+ unique_pCode)
  df_model1_data=pd.DataFrame()
  df_model1_label=pd.DataFrame()
  
  #Extracting all rows having same specific left parent code, suppose 'H'
  
  for row in range(df_llr.shape[0]):
     if df_llr.iloc[row,0] == unique_pCode:
           df_model1_data=df_model1_data.append(processedData.iloc[row],ignore_index=True)
           df_model1_label=df_model1_label.append(processedLabel.iloc[row],ignore_index=True)
  
  layer1 = 15
  validation_splitt=0 
  Activation_function = tf.nn.relu 
  loss_function = 'binary_crossentropy'
  optimizer_used = 'Adam'
  no_epochs= 500

  import tensorflow as tf
  from tensorflow import nn
  nodes_in_output_layer=df_model1_label.shape[1]
  input_size=df_model1_data.shape[1]

  model=tf.keras.models.Sequential()

  model.add(tf.keras.layers.Dense(layer1,activation=Activation_function,input_dim=input_size))
  model.add(tf.keras.layers.Dense(nodes_in_output_layer, activation=tf.nn.softmax))
                                                                             # Optimizer: Algorithm to minimize the error/loss by updating weights to improve the training of the model. e.g.
  model.compile(optimizer=optimizer_used, loss=loss_function, metrics=['accuracy'])
  #model.summary()
  history=model.fit(df_model1_data, df_model1_label, epochs= no_epochs,verbose=0)
  print(model.evaluate(df_model1_data,df_model1_label))
  #getting weights and appending to new dataset pAttributes
  temp=[unique_pCode]
  temp+=np.ndarray.tolist(np.ndarray.flatten(model.layers[0].get_weights()[0]))
  pAttributes.append(temp)
        
                                  

For unique code: H
702/702 [==============================] - 0s 279us/sample - loss: 0.0935 - acc: 0.9599
[0.0934923812959269, 0.95990026]
For unique code: R
5600/5600 [==============================] - 1s 112us/sample - loss: 0.1220 - acc: 0.9543
[0.12203805106026785, 0.95432174]
For unique code: M
2810/2810 [==============================] - 0s 147us/sample - loss: 0.0924 - acc: 0.9601
[0.09237912658376626, 0.9601248]
For unique code: I
2111/2111 [==============================] - 0s 190us/sample - loss: 0.1010 - acc: 0.9567
[0.10100604839400622, 0.95667934]
For unique code: F
3811/3811 [==============================] - 1s 146us/sample - loss: 0.1223 - acc: 0.9583
[0.12232743871865764, 0.95825243]
For unique code: T
2227/2227 [==============================] - 0s 174us/sample - loss: 0.1064 - acc: 0.9572
[0.10639931802852409, 0.9571844]
For unique code: L
664/664 [==============================] - 0s 381us/sample - loss: 0.1026 - acc: 0.9575
[0.10257285555084068, 0.9575302]
For uni

In [0]:
import csv


with open('pAttributes.csv', 'w') as csvFile:
    writer = csv.writer(csvFile)
    writer.writerows(pAttributes)

csvFile.close()

In [0]:
from google.colab import files
files.download('pAttributes.csv') 

## Model 2: Embeddings of child codes


Converting data to one Hot Encoding


In [0]:
processedData=pd.get_dummies(df_expanded.iloc[:,0:2])
processedLabel=pd.get_dummies(df_expanded.iloc[:,2:4])
processedData.shape
#processedLabel.shape

In [0]:
import tensorflow as tf
import numpy as np
from tensorflow import nn
cAttributes=[]
for unique_pCode in unique_l_pCode:
  l_code_data=pd.DataFrame()
  r_code_label=pd.DataFrame()

  #Extracting all rows having same specific left parent code, suppose 'H'

  for row in range(df_expanded.shape[0]):
     if df_expanded.iloc[row,0] == unique_pCode:
           l_code_data=l_code_data.append(df_expanded.iloc[row,0:2],ignore_index=True)
           r_code_label=r_code_label.append(df_expanded.iloc[row,2:4],ignore_index=True)

  unique_l_cCode=l_code_data.iloc[:,0].drop_duplicates()
  print("Unique Child Codes for "+ unique_pCode+ ": "+ str(unique_l_cCode.shape[0]))

  for unique_cCode in unique_l_cCode:
      print("For unique pcode: "+ unique_pCode+", unique ccode: "+ unique_cCode)
      df_model2_data=pd.DataFrame()
      df_model2_label=pd.DataFrame()

      #Extracting all rows having same specific left parent code, suppose 'H'

      for row in range(df_expanded.shape[0]):
         if df_expanded.iloc[row,0] == unique_pCode and df_expanded.iloc[row,1]==unique_cCode:
               df_model2_data=df_model2_data.append(processedData.iloc[row],ignore_index=True)
               df_model2_label=df_model2_label.append(processedLabel.iloc[row],ignore_index=True)

      print(df_model2_label.shape)
      layer1 = 15
      validation_splitt=0 
      Activation_function = tf.nn.relu 
      loss_function = 'binary_crossentropy'
      optimizer_used = 'Adam'
      no_epochs= 200

      import tensorflow as tf
      from tensorflow import nn
      nodes_in_output_layer=df_model2_label.shape[1]
      input_size=df_model2_data.shape[1]

      model=tf.keras.models.Sequential()

      model.add(tf.keras.layers.Dense(layer1,activation=Activation_function,input_dim=input_size))
      model.add(tf.keras.layers.Dense(nodes_in_output_layer, activation=tf.nn.softmax))                                                                               
      model.compile(optimizer=optimizer_used, loss=loss_function, metrics=['accuracy'])
      #model.summary()
      history=model.fit(df_model2_data, df_model2_label, epochs= no_epochs,verbose=0,validation_split=0)
      print(model.evaluate(df_model2_data,df_model2_label))
      #getting weights and appending to new dataset pAttributes
      temp=[unique_pCode,unique_cCode]
      temp+=np.ndarray.tolist(np.ndarray.flatten(model.layers[0].get_weights()[0]))
      cAttributes.append(temp)


In [0]:
import csv


with open('cAttributes.csv', 'w') as csvFile:
    writer = csv.writer(csvFile)
    writer.writerows(cAttributes)

csvFile.close()

In [0]:
from google.colab import files
files.download('cAttributes.csv') 

In [0]:
unique_pCode='H'
l_code_data=pd.DataFrame()
r_code_label=pd.DataFrame()

#Extracting all rows having same specific left parent code, suppose 'H'

for row in range(df_expanded.shape[0]):
   if df_expanded.iloc[row,0] == unique_pCode:
         l_code_data=l_code_data.append(df_expanded.iloc[row,0:2],ignore_index=True)
         r_code_label=r_code_label.append(df_expanded.iloc[row,2:4],ignore_index=True)
         
unique_l_cCode=l_code_data.iloc[:,0].drop_duplicates()
print("Unique Child Codes for "+ unique_pCode+ ": "+ str(unique_l_cCode.shape[0]))

for unique_cCode in unique_l_cCode:
  print("For unique pcode: "+ unique_pCode+", unique ccode: "+ unique_cCode)
  df_model2_data=pd.DataFrame()
  df_model2_label=pd.DataFrame()

  #Extracting all rows having same specific left parent code, suppose 'H'

  for row in range(df_expanded.shape[0]):
     if df_expanded.iloc[row,0] == unique_pCode and df_expanded.iloc[row,1]==unique_cCode:
           df_model2_data=df_model2_data.append(processedData.iloc[row],ignore_index=True)
           df_model2_label=df_model2_label.append(processedLabel.iloc[row],ignore_index=True)

  print(df_model2_data.shape)

Unique Child Codes for H: 131
For unique pcode: H, unique ccode: 269
(165, 1310)
For unique pcode: H, unique ccode: 921
(9, 1310)
For unique pcode: H, unique ccode: 813
(2, 1310)
For unique pcode: H, unique ccode: 532
(7, 1310)
For unique pcode: H, unique ccode: 348
(3, 1310)
For unique pcode: H, unique ccode: 669
(10, 1310)
For unique pcode: H, unique ccode: 250
(20, 1310)
For unique pcode: H, unique ccode: 18
(1, 1310)
For unique pcode: H, unique ccode: 905
(1, 1310)
For unique pcode: H, unique ccode: 653
(7, 1310)
For unique pcode: H, unique ccode: 612
(6, 1310)
For unique pcode: H, unique ccode: 42
(11, 1310)
For unique pcode: H, unique ccode: 41
(6, 1310)
For unique pcode: H, unique ccode: 28
(25, 1310)
For unique pcode: H, unique ccode: 919
(10, 1310)
For unique pcode: H, unique ccode: 358
(12, 1310)
For unique pcode: H, unique ccode: 504
(1, 1310)
For unique pcode: H, unique ccode: 259
(14, 1310)
For unique pcode: H, unique ccode: 332
(13, 1310)
For unique pcode: H, unique ccode

AttributeError: ignored

In [0]:
#@title Parameters for the model { run: "auto" } 

import tensorflow as tf
#run:auto command makes the code block run in real time. Whenever the user amends the input, the block runs automatically for the new set of input.

# No. of nodes in hidden layer 1
layer1 = 40 #@param {type:"integer"}

validation_splitt=0 #@param{type:"slider",min:0.000,max:1.000,step:0.05}
Activation_function = tf.nn.relu #@param ["relu", "relu6", "crelu", "elu", "selu", "softplus", "softsign", "dropout", "bias_add", "sigmoid", "tanh"]
loss_function = 'kullback_leibler_divergence' #@param ["categorical_crossentropy", "sparse_categorical_crossentropy", "binary_crossentropy","kullback_leibler_divergence","poisson","cosine_proximity"]
optimizer_used = 'Adam' #@param ["SGD", "RMSprop", "Adagrad","Adadelta","Adam","Adamax","Nadam","TFOptimizer"]

In [0]:
#@title Parametres for the model { run: "auto" }
no_epochs=302 #@param{type:"slider",min:0,max:10000,step:200}

import tensorflow as tf
from tensorflow import nn
nodes_in_output_layer=processedLabel.shape[1]
input_size=processedData.shape[1]

model=tf.keras.models.Sequential()

model.add(tf.keras.layers.Dense(layer1,activation=Activation_function,input_dim=input_size))
model.add(tf.keras.layers.Dense(nodes_in_output_layer, activation=tf.nn.softmax))
                                                                           # Optimizer: Algorithm to minimize the error/loss by updating weights to improve the training of the model. e.g.
model.compile(optimizer=optimizer_used, loss=loss_function, metrics=['accuracy'])
model.summary()
history=model.fit(processedData, processedLabel, epochs= no_epochs)


Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 40)                52440     
_________________________________________________________________
dense_1 (Dense)              (None, 20)                820       
Total params: 53,260
Trainable params: 53,260
Non-trainable params: 0
_________________________________________________________________
Instructions for updating:
Use tf.cast instead.
Epoch 1/302
32012/32012 [==============================] - 5s 169us/sample - loss: 2.0557 - acc: 0.3856
Epoch 2/302
32012/32012 [==============================] - 5s 150us/sample - loss: 1.7440 - acc: 0.4430
Epoch 3/302
32012/32012 [==============================] - 5s 151us/sample - loss: 1.6999 - acc: 0.4510
Epoch 4/302
32012/32012 [==============================] - 5s 151us/sample - loss: 1.6744 - acc: 0.457

KeyboardInterrupt: ignored

##Model


In [0]:

import numpy as np
for df_code in df_sorted:

  processedData=np.array(processedData) #direct codes converted to binary freq
  processedLabel=np.array(processedLabel)



In [0]:
#@title Parameters for the model { run: "auto" } 

import tensorflow as tf
#run:auto command makes the code block run in real time. Whenever the user amends the input, the block runs automatically for the new set of input.
no_of_layers= 3 #@param {type:"slider", min:0, max:10, step:1}
# A is the list of nodes_in_each_hidden_layer for each of the user defined hidden layers. Eg. A=[128,256,64] implies 3 hidden layers with 128 nodes, 256 nodes and 64 nodes respectively starting from the input layer to the output layer.
A=[]
# No. of nodes in hidden layer 1
nodes_in_layer1 = 300 #@param {type:"integer"}

validation_splitt=0 #@param{type:"slider",min:0.000,max:1.000,step:0.05}
Activation_function = tf.nn.crelu #@param ["relu", "relu6", "crelu", "elu", "selu", "softplus", "softsign", "dropout", "bias_add", "sigmoid", "tanh"]
loss_function = 'kullback_leibler_divergence' #@param ["categorical_crossentropy", "sparse_categorical_crossentropy", "binary_crossentropy","kullback_leibler_divergence","poisson","cosine_proximity"]
optimizer_used = 'Adam' #@param ["SGD", "RMSprop", "Adagrad","Adadelta","Adam","Adamax","Nadam","TFOptimizer"]

In [0]:
#@title Parametres for the model { run: "auto" }

#no_epochs is the integer input for total no. of model parsing -
#from input to generating output and back to updating the weights -
#according to the error for all training examples.

no_epochs=302 #@param{type:"slider",min:0,max:10000,step:200}
 
import time
start=time.time()

#Libraries for current codeblock
import tensorflow as tf
from tensorflow import nn
nodes_in_output_layer=processedLabel.shape[1]
input_size=processedData.shape[1]

model=tf.keras.models.Sequential()

#Various types of layers in keras: Dense, Flatten, Input, Permute, Reshape.[3]
#Dense implements the operation: output = activation(dot(input, kernel) + bias)[3]

#Adding input layer with size of input i.e. 10 in this case. 
#The first parameter for function Dense() is No. of nodes. in the layer. model.add appends the newly created hidden layer to the model.

model.add(tf.keras.layers.Dense(A[0],activation=Activation_function,input_dim=input_size))
for layer_iterator in range(1,len(A)-1):
  model.add(tf.keras.layers.Dense(A[layer_iterator+1], activation=Activation_function))
#Creating dropout layer
#model.add(tf.keras.layers.Dropout(rate=dropout_rate))
# Creating the output layer with 4 nodes. Since Output layer 
model.add(tf.keras.layers.Dense(nodes_in_output_layer, activation=tf.nn.softmax))

# Parameters for model creation i.e. optimizers, loss function
                                                                                # metric: Accuracy. Accuracy is the measure of correctness of the model
                                                                                # - , basically rate to which it classifies the inputs correctly.
                                                                                # Validation Accuracy: Percentage of correctly classifying on input chosen from the train dataset itself.
                                                                                # Testing Accuracy:  Percentage of correctly classifying on input from a new dataset following the same problem as the train dataset.

                                                                                # Optimizer: Algorithm to minimize the error/loss by updating weights to improve the training of the model. e.g.
model.compile(optimizer=optimizer_used, loss=loss_function, metrics=['accuracy'])
model.summary()
history=model.fit(processedData, processedLabel, epochs= no_epochs)

end=time.time()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_15 (Dense)             (None, 16)                40        
_________________________________________________________________
dense_16 (Dense)             (None, 16)                136       
_________________________________________________________________
dense_17 (Dense)             (None, 2)                 34        
Total params: 210
Trainable params: 210
Non-trainable params: 0
_________________________________________________________________
Epoch 1/302
27492/27492 [==============================] - 4s 146us/sample - loss: -0.0801 - acc: 0.4539
Epoch 2/302
27492/27492 [==============================] - 4s 140us/sample - loss: -0.0844 - acc: 0.4502
Epoch 3/302
27492/27492 [==============================] - 4s 140us/sample - loss: -0.0851 - acc: 0.4617
Epoch 4/302
 5696/27492 [=====>........................] - ETA: 3s - loss: -0.0856 - acc: 0.4

KeyboardInterrupt: ignored

In [0]:
loss,acc=model.evaluate(processedData,processedLabel)
print("Loss=",loss,"\nAccuracy=",acc)

%matplotlib inline
df = pd.DataFrame(history.history)
df.plot(subplots=True, grid=True, figsize=(10,15))
print(np.std(df))

27492/27492 [==============================] - 1s 49us/sample - loss: -0.0861 - acc: 0.4559
Loss= -0.08612929332495244 
Accuracy= 0.45587808


NameError: ignored

In [0]:
predictedTestLabel = []
for i,j in zip(processedData,processedLabel):
    y = model.predict(np.array(i).reshape(-1,5))
    predictedTestLabel.append(y.argmax())
def decodeLabel(encodedLabel):
    if encodedLabel == 0:
        return "a"
    elif encodedLabel == 1:
        return "b"
    elif encodedLabel == 2:
        return "c"
    elif encodedLabel == 3:
        return "d"
    elif encodedLabel == 4:
        return "e"

predictedTestLabel2=[]
for Label in predictedTestLabel:
  predictedTestLabel2.append(decodeLabel(Label))
predictedTestLabel2=pd.DataFrame(predictedTestLabel2)
print(pd.concat([df_adj,predictedTestLabel2],ignore_index=True))

ValueError: ignored

In [0]:
pd.DataFrame(predictedTestLabel2)

,0
0,b
1,a
2,b
3,b
4,b
5,a
6,b
7,b
8,b
9,a
